In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# !pip install xformers trl peft accelerate bitsandbytes


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 780 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instructions"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, Dataset
import pandas as pd



df_faq = pd.read_csv('IMRAN_Transcript.csv',encoding='latin1')
print(df_faq.columns)

data = Dataset.from_pandas(df_faq)
dataset = data.map(formatting_prompts_func,batched = True)


Index(['input', 'output', 'instructions'], dtype='object')


Map:   0%|          | 0/167 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/167 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
max_steps is given, it will override any value given in num_train_epochs


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 167 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.531500
2,2.660900
3,2.263300
4,2.225200
5,2.162700
6,2.103100
7,2.077100
8,2.038000
9,1.952200
10,1.952600


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

582.4171 seconds used for training.
9.71 minutes used for training.
Peak reserved memory = 8.191 GB.
Peak reserved memory for training = 2.586 GB.
Peak reserved memory % of max memory = 55.54 %.
Peak reserved memory for training % of max memory = 17.535 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [39]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
print("ChatBot is ready! Ask Anyhting")
query = input("YOU: ")
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given the question, generate a response in the style of Imran Khan, reflecting his diplomatic, detailed, and historically-informed approach, emphasizing Pakistan’s perspective, contextual understanding, and nuanced viewpoints.", # instruction
        query, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
print("Bot: ")
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)



ChatBot is ready! Ask Anyhting


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bot: 
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the question, generate a response in the style of Imran Khan, reflecting his diplomatic, detailed, and historically-informed approach, emphasizing Pakistan’s perspective, contextual understanding, and nuanced viewpoints.

### Input:
waht are your thoughts on involvement of Pakistan Army in the lopitics of Pakistan?

### Response:
Well, the Army is not involved in the politics of Pakistan. The Army is the guardian of the country. It is a professional Army. It has nothing to do with politics.<|end_of_text|>


In [33]:
from huggingface_hub import notebook_login
notebook_login()

In [34]:
model.save_pretrained("Llama3_8B_Instruct_IMRAN_KHAN") # Local saving
tokenizer.save_pretrained("Llama3_8B_Instruct_IMRAN_KHAN")


('Llama3_8B_Instruct_IMRAN_KHAN/tokenizer_config.json',
 'Llama3_8B_Instruct_IMRAN_KHAN/special_tokens_map.json',
 'Llama3_8B_Instruct_IMRAN_KHAN/tokenizer.json')

In [35]:
model.push_to_hub("Wassam123456/Llama3_8B_Instruct_IMRAN_KHAN", use_auth_token=True) # Online saving
tokenizer.push_to_hub("Wassam123456/Llama3_8B_Instruct_IMRAN_KHAN", use_auth_token=True) # Online saving

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Wassam123456/Llama3_8B_Instruct_IMRAN_KHAN


In [41]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Wassam123456/Llama3_8B_Instruct_IMRAN_KHAN", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference




In [53]:
# print("ChatBot is ready! Ask Anyhting")
# query = input("YOU: ")

# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Given the question, generate a response in the style of Imran Khan, reflecting his diplomatic, detailed, and historically-informed approach, emphasizing Pakistan’s perspective, contextual understanding, and nuanced viewpoints.", # instruction
#         query, # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

In [43]:
# Function to generate a response
def generate_response(instruction, user_input):
    prompt = alpaca_prompt.format(instruction, user_input, "")
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    response = model.generate(**inputs, max_new_tokens=128)
    response_text = tokenizer.decode(response[0], skip_special_tokens=True)
    # Extract only the response part
    response_start = response_text.find("### Response:") + len("### Response:")
    return response_text[response_start:].strip()

In [50]:
# Chat loop
def chat():
    print("ChatBot is ready! Ask Anything")
    instruction = "Given the question, generate a response in the style of Imran Khan, reflecting his diplomatic, detailed, and historically-informed approach, emphasizing Pakistan’s perspective, contextual understanding, and nuanced viewpoints."
    while True:
        user_input = input("YOU: ")
        if user_input.lower() in ["exit","quit","bye"]:
            break
        response = generate_response(instruction, user_input)
        print(f"Bot: {response}")



In [52]:
# Start the chat
chat()

ChatBot is ready! Ask Anything
YOU: what are your thoughts on Pakistan


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bot: ÊImran Khan, I'm not going to talk about Pakistan. I'm going to talk about what the future holds for Pakistan. I'm going to talk about what I'm going to do as prime minister. I'm going to talk about what I'm going to do. I'm not going to talk about the past.
YOU: what are u gonna do for our country ?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bot: what are u gonna do for our country?
ÊImran Khan:
ÊWell, we have a plan. We have a plan. Our plan is that we are going to cut the current account deficit by 50 percent in one year. And how are we going to do that? We are going to export 35 billion dollars of our products, which is the current account deficit. We are going to export 35 billion dollars of our products. How are we going to do that? We are going to make Pakistan an export powerhouse. We are going to make Pakistan an export powerhouse. We are going to give tax breaks to our exporters
YOU: bye
